<a href="https://colab.research.google.com/github/bsekiewicz/dw_competition/blob/master/winner_draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install scikit-plot
!pip install eli5
!pip instapgrade tables

     |████████████████████████████████| 112kB 36.6MB/s 
     |████████████████████████████████| 4.3MB 43.5MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


In [0]:
cd '/content/drive/My Drive/Colab Notebooks/competition/data/'

/content/drive/My Drive/Colab Notebooks/competition/data


In [0]:
import pandas as pd # from pandas.core.common import flatten
import numpy as np
import string
import regex
import gc

from collections import defaultdict 

from sklearn.model_selection import cross_val_score, StratifiedKFold, GroupKFold, train_test_split
from sklearn.metrics import mean_squared_error as mse, precision_score, recall_score
from sklearn import utils

from scikitplot.metrics import plot_confusion_matrix, plot_calibration_curve
from scikitplot.estimators import plot_learning_curve, plot_feature_importances

from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
import lightgbm as lgb


import eli5
from eli5.sklearn import PermutationImportance

from hyperopt import hp, fmin, tpe, STATUS_OK

#%matplotlib inline

In [0]:
np.random.seed(0)

## READ DATA

In [0]:
# train = pd.read_hdf('../input/dw-competition/train_online_retail.h5').reset_index(drop=True)
# test = pd.read_hdf('../input/dw-competition/test_online_retail.h5').reset_index(drop=True)

train = pd.read_hdf('train_online_retail.h5').reset_index(drop=True)
test = pd.read_hdf('test_online_retail.h5').reset_index(drop=True)

df = pd.concat([train, test], sort=False)

del test

In [0]:
# repair price total
df['price_total'] = df['quantity'] * df['price_unit']

In [0]:
gc.collect()  # release memory

2576

## UTILS

In [0]:
### FE

# 
def get_feats(df, out=[]):
    feats = df.select_dtypes([np.number, np.bool]).columns
    black_list = ['is_test', 'is_canceled', 'is_canceled_pred', 'total_return', 'total_return_pred', 'invoice_price_total']
    
    return [x for x in feats if x not in black_list + out]

# 
def group_by_key(df, group_key, agg_func, reversed=False):
    df_temp = df[~df['is_canceled'].isnull()]
    if reversed:
        df_temp['is_canceled'] = df_temp['is_canceled'].map(lambda x: False if x else True)
    dict_ = df_temp.groupby(group_key)['is_canceled'].agg(agg_func).to_dict()
    if -1 in dict_: del dict_[-1]
    
    return df_temp[group_key].map(lambda x: dict_.get(x, -1))

#
def group_by_key_alt(df, group_key, group_key_alt, agg_func, reversed=False):
    df_temp = df[~df['is_canceled'].isnull()]
    if reversed:
        df_temp['is_canceled'] = df_temp['is_canceled'].map(lambda x: False if x else True)
    df_temp[group_key_alt] = df_temp[group_key_alt]*df_temp['is_canceled']
    dict_ = df_temp.groupby(group_key)[group_key_alt].agg(agg_func).to_dict()
    if -1 in dict_: del dict_[-1]
    
    return df_temp[group_key].map(lambda x: dict_.get(x, -1))

#
def aggregate_by(df, group_key, agg_key, agg_name, agg_fun, filtered=False):
    if filtered:
        df_temp = df[~df['is_canceled'].isnull()]
    else:
        df_temp = df
    return pd.merge(df, df_temp.groupby(group_key).agg(agg_name=(agg_key, agg_fun)), how='left', on=[group_key]).rename(columns = {'agg_name': agg_name})

# 
def prepare_categories(df, cats):
    for c in cats:
        df['cat_' + c] = df['description'].map(lambda x: c in str(x).lower())
    return df

In [0]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520142 entries, 0 to 520141
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   invoice       520142 non-null  int32         
 1   stock_code    520142 non-null  int16         
 2   description   517903 non-null  object        
 3   quantity      520142 non-null  int32         
 4   invoice_date  520142 non-null  datetime64[ns]
 5   price_unit    520142 non-null  float16       
 6   price_total   520142 non-null  float32       
 7   customer_id   520142 non-null  int16         
 8   country       520142 non-null  object        
 9   is_canceled   520142 non-null  bool          
 10  is_test       520142 non-null  bool          
dtypes: bool(2), datetime64[ns](1), float16(1), float32(1), int16(2), int32(2), object(2)
memory usage: 21.8+ MB


## FE

In [0]:
# df = aggregate_by_invoice(df, 'price_total', 'invoice_price_total')

In [0]:
# feature engineering



#
df['cnt_customer_cancel'] = df['customer_id'].map(group_to_dict(df, 'customer_id')).astype(np.int)
df['cnt_customer_not_cancel'] = df['customer_id'].map(group_to_dict(df, 'customer_id', not_cancel=True)).astype(np.int)
df['cnt_customer_orders'] = df['customer_id'].map(group_to_dict(df, 'customer_id', agg_func=np.size)).astype(np.int)

df['cnt_product_cancel'] = df['stock_code'].map(group_to_dict(df, 'stock_code')).astype(np.int)
df['cnt_product_not_cancel'] = df['stock_code'].map(group_to_dict(df, 'stock_code', not_cancel=True)).astype(np.int)
df['cnt_product_orders'] = df['stock_code'].map(group_to_dict(df, 'stock_code', agg_func=np.size)).astype(np.int)
df['cnt_product_stock_cancel'] = df['stock_code'].map(group_to_dict_alt(df, 'stock_code')).astype(np.int)
df['cnt_product_stock_not_cancel'] = df['stock_code'].map(group_to_dict_alt(df, 'stock_code', not_cancel=True)).astype(np.int)

# repair stock code
df['description'] = df['description'].map(lambda x: str(x).lower().strip())
df['description'] = df['description'].map(lambda x: regex.sub('|\\'.join(string.punctuation), ' ', x))
df['description'] = df['description'].map(lambda x: regex.sub(' ([0-9]+[^ ]{0,} )+', ' ', x))
df['description'] = df['description'].map(lambda x: regex.sub('^[0-9]+[^ ]{0,} ', ' ', x))
df['description'] = df['description'].map(lambda x: regex.sub(' [0-9]+[^ ]{0,}$', ' ', x))
df['description'] = df['description'].map(lambda x: regex.sub('[ ]+', ' ', x).strip())
df['stock_code_2'] = df['description'].factorize()[0]

df['cnt_product_cancel_2'] = df['stock_code_2'].map(group_to_dict(df, 'stock_code_2')).astype(np.int)
df['cnt_product_not_cancel_2'] = df['stock_code_2'].map(group_to_dict(df, 'stock_code_2', not_cancel=True)).astype(np.int)
df['cnt_product_orders_2'] = df['stock_code_2'].map(group_to_dict(df, 'stock_code_2', agg_func=np.size)).astype(np.int)
df['cnt_product_stock_cancel_2'] = df['stock_code_2'].map(group_to_dict_alt(df, 'stock_code_2')).astype(np.int)
df['cnt_product_stock_not_cancel_2'] = df['stock_code_2'].map(group_to_dict_alt(df, 'stock_code_2', not_cancel=True)).astype(np.int)

# datetime
df['invoice_date_y'] = df.invoice_date.dt.year
df['invoice_date_m'] = df.invoice_date.dt.month
df['invoice_date_d'] = df.invoice_date.dt.day
df['invoice_date_h'] = df.invoice_date.dt.hour
df['invoice_date_min'] = df.invoice_date.dt.minute
df['invoice_date_dow'] = df.invoice_date.dt.dayofweek

# countries
df['country_europe'] = df['country'].map(lambda x: 0 if x in ['United Kingdom', 'France', 'EIRE', 'Netherlands', 'Poland', 'Germany', 'Spain', 
                                                              'Channel Islands', 'Italy', 'Cyprus', 'Greece', 'Norway', 'Portugal', 'Denmark', 
                                                              'Sweden', 'Finland', 'Belgium', 'Malta', 'Austria', 'Switzerland', 'Lithuania', 
                                                              'Iceland','Czech Republic'] else (-1 if x == 'Unspecified' else 1))
df['country_cat'] = df['country'].factorize()[0]

df['description_len'] = df['description'].map(lambda x: len(x))
df['description_cnt_space'] = df['description'].map(lambda x: x.count(' '))

# aggregations
df = aggregate_by_invoice(df, 'price_unit', 'price_unit_min', agg_fun=np.min)
df = aggregate_by_invoice(df, 'price_unit', 'price_unit_max', agg_fun=np.max)
df = aggregate_by_invoice(df, 'price_unit', 'price_unit_mean', agg_fun=np.mean)
df = aggregate_by_invoice(df, 'price_unit', 'price_unit_median', agg_fun=np.median)
df = aggregate_by_invoice(df, 'price_unit', 'price_unit_sum', agg_fun=np.sum)
df = aggregate_by_invoice(df, 'quantity', 'quantity_min', agg_fun=np.min)
df = aggregate_by_invoice(df, 'quantity', 'quantity_max', agg_fun=np.max)
df = aggregate_by_invoice(df, 'quantity', 'quantity_mean', agg_fun=np.mean)
df = aggregate_by_invoice(df, 'quantity', 'quantity_median', agg_fun=np.median)
df = aggregate_by_invoice(df, 'quantity', 'quantity_sum', agg_fun=np.sum)



df = aggregate_by(df, 'quantity', 'quantity_stock', 'stock_code', agg_fun=np.sum)

In [0]:
# transformations

df['price_total_log'] = df.price_total.map(lambda x: -1 if x<0 else np.log1p(x))
df['price_unit_sum_log'] = df.price_unit_sum.map(lambda x: -1 if x<0 else np.log1p(x))
df['quantity_sum_log'] = df.quantity_sum.map(lambda x: -1 if x<0 else np.log1p(x))
df['quantity_log'] = df.quantity.map(lambda x: -1 if x<0 else np.log1p(x))
df['quantity_stock_log'] = df.quantity_stock.map(lambda x: -1 if x<0 else np.log1p(x))
df['invoice_price_total_log'] = df.invoice_price_total.map(lambda x: -1 if x<0 else np.log1p(x))
df['cnt_product_stock_cancel_log'] = df.cnt_product_stock_cancel.map(lambda x: -1 if x<0 else np.log1p(x))
df['cnt_product_stock_not_cancel_log'] = df.cnt_product_stock_not_cancel.map(lambda x: -1 if x<0 else np.log1p(x))
df['cnt_product_stock_cancel_2_log'] = df.cnt_product_stock_cancel_2.map(lambda x: -1 if x<0 else np.log1p(x))
df['cnt_product_stock_not_cancel_2_log'] = df.cnt_product_stock_not_cancel_2.map(lambda x: -1 if x<0 else np.log1p(x))
df['cnt_customer_cancel_log'] = df.cnt_customer_cancel.map(lambda x: -1 if x<0 else np.log1p(x))

out = ['cnt_customer_cancel','price_total','price_unit_sum','quantity_sum','quantity','quantity_stock','invoice_price_total','cnt_product_stock_cancel','cnt_product_stock_not_cancel','cnt_product_stock_cancel_2','cnt_product_stock_not_cancel_2',]

In [0]:
# expensive or cheap
tmp = df.copy()
tmp = tmp[~tmp['is_canceled'].isnull()]
dict_median = tmp.groupby('description')['price_unit'].agg(np.median).to_dict()
dict_mean = tmp.groupby('description')['price_unit'].agg(np.mean).to_dict()

if '' in dict_median: del dict_median['']
if '' in dict_mean: del dict_mean['']

t = list(dict_median.keys())
for d in t:
    if dict_median[d] == 0:
        del dict_median[d]

t = list(dict_mean.keys())
for d in t:
    if dict_mean[d] == 0:
        del dict_mean[d]

def ftmp(x, dict_):
  v = (x['price_unit'] - dict_[x['description']])/dict_[x['description']]
  if v <= -1:
    return 0
  elif v <= 0.25:
    return 1
  elif v >= 1:
    return 4
  elif v >= 0.25:
    return 3
  else:
    return 2

# df['price_cheap_exp_median'] = df.apply(lambda x: -1 if x['description'] not in dict_median else ftmp(x, dict_median), axis=1)
# df['price_cheap_exp_mean'] = df.apply(lambda x: -1 if x['description'] not in dict_mean else ftmp(x, dict_mean), axis=1)
df['price_cheap_exp_median'] = df.apply(lambda x: 0 if x['description'] not in dict_median else (x['price_unit'] - dict_median[x['description']])/dict_median[x['description']], axis=1)
df['price_cheap_exp_mean'] = df.apply(lambda x: 0 if x['description'] not in dict_mean else (x['price_unit'] - dict_mean[x['description']])/dict_mean[x['description']], axis=1)

In [0]:
# # description tokenizer

# desc = df[~df['is_canceled'].isnull()]['description']
# desc = list(set(flatten([x.split(' ') for x in desc])))
# desc = [x for x in desc if len(x) > 2]

# def find_tokens(data, token):
#     tmp = data.copy()
#     tmp['is_token'] = tmp['description'].map(lambda x: token in x)

#     x = tmp.groupby('is_token')['is_canceled'].agg(np.sum)
#     y = tmp.groupby('is_token')['is_canceled'].agg(np.size)
    
#     freq = (x[1]+y[1])/(x[0]+y[0])
#     diff = list(x/y)
    
# #     print('{}: freq = {} | diff = {}'.format(c, freq, diff[0]/diff[1]))
    
#     if diff[1] == 0:
#         return freq, 100
#     else:
#         return freq, diff[0]/diff[1]
    
# train = df.copy()
# train = train[ ~train['is_canceled'].isnull()][['invoice', 'is_canceled', 'description']]

In [0]:
# find_tokens(train, 'pink')

In [0]:
# cat_desc = []
# for c in sorted(desc):
#     freq, diff = find_tokens(train, c)
#     if freq > 0.02:
#         if abs(diff-1) > 0.5:
#             cat_desc += [c]
#             print('{}: freq = {} | diff = {}'.format(c, freq, diff))

In [0]:
cat_desc = [
    'bottle',
    'car',
    'case',
    'christmas',
    'craft',
    'decoration',
    'door',
    'felt',
    'glass',
    'hot',
    'love',
    'mat',
    'metal',
    'pack',
    'paper',
    'water',
]

In [0]:
df = prepare_categories(df, cat_desc)

In [0]:
# TODO: 
# colors
# size

In [0]:
df.sample(10)

,invoice,stock_code,description,quantity,invoice_date,price_unit,price_total,customer_id,country,is_canceled,is_test,cnt_customer_cancel,cnt_customer_not_cancel,cnt_customer_orders,cnt_product_cancel,cnt_product_not_cancel,cnt_product_orders,cnt_product_stock_cancel,cnt_product_stock_not_cancel,stock_code_2,cnt_product_cancel_2,cnt_product_not_cancel_2,cnt_product_orders_2,cnt_product_stock_cancel_2,cnt_product_stock_not_cancel_2,invoice_date_y,invoice_date_m,invoice_date_d,invoice_date_h,invoice_date_min,invoice_date_dow,country_europe,country_cat,description_len,description_cnt_space,price_unit_min,price_unit_max,price_unit_mean,price_unit_median,price_unit_sum,quantity_min,quantity_max,quantity_mean,quantity_median,quantity_sum,invoice_price_total,quantity_stock,price_total_log,price_unit_sum_log,quantity_sum_log,quantity_log,quantity_stock_log,invoice_price_total_log,cnt_product_stock_cancel_log,cnt_product_stock_not_cancel_log,cnt_product_stock_cancel_2_log,cnt_product_stock_not_cancel_2_log,cnt_customer_cancel_log,price_cheap_exp_median,price_cheap_exp_mean,cat_bottle,cat_car,cat_case,cat_christmas,cat_craft,cat_decoration,cat_door,cat_felt,cat_glass,cat_hot,cat_love,cat_mat,cat_metal,cat_pack,cat_paper,cat_water
237123,26748,4530,christmas lights reindeer,12,2010-11-23 12:04:00,8.500000,102.000000,2463,United Kingdom,False,False,0,98,98,13,265,278,42,1165,4180,13,265,278,42,1165,2010,11,23,12,4,1,0,0,25,2,0.209961,9.953125,2.404297,1.450195,134.6250,2,48,15.000000,12.0,840,1108.835938,1207.0,4.634729,4.909894,6.734592,2.564949,7.096721,7.011967,3.761200,7.061334,3.761200,7.061334,0.000000,0.000000,-0.098592,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
607839,9972,680,lavender scented fabric heart,10,2010-04-28 14:59:00,1.250000,12.500000,1595,France,NaN,True,0,18,18,4,259,263,45,2775,480,4,259,263,45,2775,2010,4,28,14,59,2,0,1,29,3,1.250000,18.000000,5.187500,3.750000,57.0625,2,10,5.818182,6.0,64,249.671875,2820.0,2.602690,4.061520,4.174387,2.397895,7.944847,5.524145,3.828641,7.928766,3.828641,7.928766,0.000000,0.000000,-0.306984,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
14451,1398,13,love building block word,1,2009-12-13 15:54:00,5.949219,5.949219,831,United Kingdom,False,False,2,241,243,11,879,890,29,2877,1,11,879,890,29,2877,2009,12,13,15,54,6,0,0,24,3,0.189941,7.949219,2.541016,1.800781,162.6250,1,25,2.390625,1.0,153,213.858398,2906.0,1.938629,5.097577,5.036953,0.693147,7.974877,5.369979,3.401197,7.964851,3.401197,7.964851,1.098612,0.000000,-0.108836,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
216632,24874,3955,childs garden trowel pink,1,2010-11-07 10:59:00,0.850098,0.850098,2116,United Kingdom,False,False,1,225,226,1,180,181,2,1108,3379,1,180,181,2,1108,2010,11,7,10,59,6,0,0,25,3,0.290039,8.953125,2.316406,1.650391,113.5000,1,32,5.367347,3.0,263,517.296875,1110.0,0.615238,4.740575,5.575949,0.693147,7.013016,6.250548,1.098612,7.011214,1.098612,7.011214,0.693147,0.000000,-0.093229,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
520512,60,562,door mat welcome puppies,2,2009-12-01 12:30:00,6.750000,13.500000,49,EIRE,NaN,True,39,2003,2042,5,343,348,10,2179,429,1,146,147,2,630,2009,12,1,12,30,1,0,4,24,3,0.549805,14.953125,4.523438,3.449219,149.2500,1,24,6.424242,6.0,212,584.488281,2189.0,2.674149,5.012301,5.361292,1.098612,7.691657,6.372446,2.397895,7.687080,1.098612,6.447306,3.688879,-0.098592,-0.168431,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False
56699,6296,226,blue piece mini dots cutlery set,1,2010-03-04 12:24:00,3.750000,3.750000,66,United Kingdom,False,False,53,1236,1289,4,489,493,80,3929,161,2,257,259,73,1984,2010,3,4,12,24,3,0,0,32,5,0.099976,10.953125,2.759766,2.099609,165.6250,1,9,2.850000,2.0,171,331.582642,4009.0,1.558145,5.115746,5.147494,0.693147,8.296547,5.8

In [0]:
# prepare to train
train = df[~df['is_canceled'].isnull()].copy()
test = df[df['is_canceled'].isnull()].copy()

In [0]:
train['is_canceled'] = train['is_canceled'].astype('bool')

In [0]:
# release memory
del df
gc.collect()

11

In [0]:
train = train[~train.invoice.isin([53588, 53587, 30479])]

In [0]:
train = train[train.description != 'amazon fee']

In [0]:
train = train[train.description != 'adjust bad debt']

In [0]:
train.sort_values(by='invoice_price_total', ascending=False).head(10)

,invoice,stock_code,description,quantity,invoice_date,price_unit,price_total,customer_id,country,is_canceled,is_test,cnt_customer_cancel,cnt_customer_not_cancel,cnt_customer_orders,cnt_product_cancel,cnt_product_not_cancel,cnt_product_orders,cnt_product_stock_cancel,cnt_product_stock_not_cancel,stock_code_2,cnt_product_cancel_2,cnt_product_not_cancel_2,cnt_product_orders_2,cnt_product_stock_cancel_2,cnt_product_stock_not_cancel_2,invoice_date_y,invoice_date_m,invoice_date_d,invoice_date_h,invoice_date_min,invoice_date_dow,country_europe,country_cat,description_len,description_cnt_space,price_unit_min,price_unit_max,price_unit_mean,price_unit_median,price_unit_sum,quantity_min,quantity_max,quantity_mean,quantity_median,quantity_sum,invoice_price_total,quantity_stock,price_total_log,price_unit_sum_log,quantity_sum_log,quantity_log,quantity_stock_log,invoice_price_total_log,cnt_product_stock_cancel_log,cnt_product_stock_not_cancel_log,cnt_product_stock_cancel_2_log,cnt_product_stock_not_cancel_2_log,cnt_customer_cancel_log,price_cheap_exp_median,price_cheap_exp_mean,cat_bottle,cat_car,cat_case,cat_christmas,cat_craft,cat_decoration,cat_door,cat_felt,cat_glass,cat_hot,cat_love,cat_mat,cat_metal,cat_pack,cat_paper,cat_water
228272,25892,4433,alarm clock bakelike orange,6,2010-11-15 16:02:00,8.953125,53.718750,-1,United Kingdom,False,False,1,81,83,10,312,322,16,1581,3990,10,312,322,16,1581,2010,11,15,16,2,0,0,0,27,3,1.0,24.953125,5.097656,3.949219,565.5,4,902,120.603604,73.0,13387,49840.712891,1597.0,4.002206,6.339477,9.502114,1.945910,7.376508,10.816608,2.833213,7.366445,2.833213,7.366445,0.693147,1.387500,0.994778,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
228294,25892,4298,christmas musical zinc star,21,2010-11-15 16:02:00,1.950195,40.954102,-1,United Kingdom,False,False,1,81,83,1,84,85,24,1169,3771,1,84,85,24,1169,2010,11,15,16,2,0,0,0,27,3,1.0,24.953125,5.097656,3.949219,565.5,4,902,120.603604,73.0,13387,49840.712891,1193.0,3.736576,6.339477,9.502114,3.091042,7.085064,10.816608,3.218876,7.064759,3.218876,7.064759,0.693147,1.349412,1.545570,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
228266,25892,4448,polkadot pen,76,2010-11-15 16:02:00,1.950195,148.214844,-1,United Kingdom,False,False,1,81,83,0,157,157,0,6832,4083,0,142,142,0,6083,2010,11,15,16,2,0,0,0,12,1,1.0,24.953125,5.097656,3.949219,565.5,4,902,120.603604,73.0,13387,49840.712891,6832.0,5.005387,6.339477,9.502114,4.343805,8.829519,10.816608,0.000000,8.829519,0.000000,8.713418,0.693147,1.294084,0.792639,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
228267,25892,4388,ribbon reel snowy village,109,2010-11-15 16:02:00,3.949219,430.464844,-1,United Kingdom,False,False,1,81,83,2,259,261,20,2444,3900,2,259,261,20,2444,2010,11,15,16,2,0,0,0,25,3,1.0,24.953125,5.097656,3.949219,565.5,4,902,120.603604,73.0,13387,49840.712891,2464.0,6.067186,6.339477,9.502114,4.700480,7.809947,10.816608,3.044522,7.801800,3.044522,7.801800,0.693147,1.392899,0.872222,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
228268,25892,4385,set of ribbons vintage christmas,130,2010-11-15 16:02:00,5.949219,773.398438,-1,United Kingdom,False,False,1,81,83,3,395,398,24,5987,3912,3,386,389,24,4776,2010,11,15,16,2,0,0,0,32,4,1.0,24.953125,5.097656,3.949219,565.5,4,902,120.603604,73.0,13387,49840.712891,6011.0,6.652087,6.339477,9.502114,4.875197,8.701513,10.816608,3.218876,8.697513,3.218876,8.471568,0.693147,1.058108,0.844942,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
228269,25892,4379,vintage christmas stickers,51,2010-11-15 16:02:00,2.949219,150.410156,-1,United Kingdom,False,False,1,81,83,0,168,168,0,1605,3892,0,156,156,0,1285,2010,11,15,16,2,0,0,0,26,2,1.0,24.953125,5.097656,3.949219,565.5,4,902,120.603604,73.0,13387,49840.712891,1605.0,5.019992,6.339477,9.502114,

In [0]:
# + valid set

train, valid = train_test_split(train, test_size=0.2)

In [0]:
# dummy model #0 - the worst case ever

orders_train = train[['invoice', 'invoice_price_total']].drop_duplicates()
orders_test = test[['invoice', 'invoice_price_total']].drop_duplicates()

mse_train = sum(orders_train.invoice_price_total * orders_train.invoice_price_total) / (len(orders_train)-1)
mse_test = sum(orders_test.invoice_price_total * orders_test.invoice_price_total) / (len(orders_test)-1)

print('MSE train vs MSE test: {} vs {} (ratio {})'.format(mse_train, mse_test, round(mse_train/mse_test*100)/100))

MSE train vs MSE test: 1266020.118589018 vs 1894213.902757539 (ratio 0.67)


In [0]:
# dummy model #1

feats = ['invoice']

orders_train = train[['invoice', 'is_canceled', 'invoice_price_total']].drop_duplicates()
orders_train['total_return'] = orders_train['invoice_price_total'] * orders_train['is_canceled'] 

X = orders_train[feats].values
y = orders_train['total_return'].values

model = DummyRegressor(strategy='mean')
model.fit(X, y)
y_pred = model.predict(X)

mse(y, y_pred)

115350.79594281623

In [0]:
115680 / 263376 # vs submission

0.4392199744851467

In [0]:
# dummy model #2

orders_train = train[['invoice', 'is_canceled', 'invoice_price_total']].drop_duplicates()
orders_train['total_return'] = orders_train['invoice_price_total'] * orders_train['is_canceled'] 

sum(orders_train.total_return * orders_train.total_return) / (len(orders_train)-1)

115705.54950087005

In [0]:
# model #1

def run_model(model, feats, data, data_to_predict=None, data_valid=None, threshold=0.02, scenario={'opt': '3', 'fun': np.mean}):
#     data = train.copy()
#     data_test = test.copy()

    X_invoice = data[['invoice', 'is_canceled']].drop_duplicates().reset_index(drop=True)
    
    scores = []
    ratio_upper = 0
    ratio_lower = 0
    preditions = []
    valid_preditions = []
    
    skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
    for train_invoice_index, test_invoice_index in skf.split(X_invoice[['invoice']], X_invoice['is_canceled'] == True):
        train_invoice_list = list(X_invoice.iloc[train_invoice_index].invoice)
        test_invoice_list = list(X_invoice.iloc[test_invoice_index].invoice)

        df_train, df_test = data[data.invoice.isin(train_invoice_list)], data[data.invoice.isin(test_invoice_list)]

        X_train, X_test = df_train[feats], df_test[feats]
        y_train, y_test = df_train['is_canceled'], df_test['is_canceled']

        # class weights
        class_weights = list(class_weight.compute_class_weight('balanced',
                                                               np.unique(df_train['is_canceled']),
                                                               df_train['is_canceled']))

        w_array = np.ones(y_train.shape[0], dtype = 'float')
        for i, val in enumerate(y_train):
            w_array[i] = class_weights[val-1]

        # model fit
        model.fit(X_train.values, y_train, eval_metric=["auc","error","logloss"], sample_weight=w_array)

#         tmp = df_train.copy()[['invoice', 'invoice_price_total', 'is_canceled']].drop_duplicates()
#         mse_score_max = sum(tmp.invoice_price_total * tmp.invoice_price_total) / (len(tmp)-1)
#         orders_train = df_train.copy()    
#         orders_train['is_canceled_pred_1'] = model.predict(X_train.values) # 0,1
#         orders_train['is_canceled_pred_2'] = model.predict_proba(X_train.values)[:, 1] # probrabilities
#         orders_train['is_canceled_pred_3'] = orders_train['is_canceled_pred_2']*orders_train['price_total']/orders_train['invoice_price_total'] # probabilities weightedby price
#         tmp = pd.merge(tmp, orders_train.groupby('invoice').agg(is_canceled_pred = ('is_canceled_pred_' + str(scenario['opt']), scenario['fun'])).reset_index(), how='left', on=['invoice'])            
#         tmp['total_return'] = tmp['is_canceled'] * tmp['invoice_price_total']
#         tmp['total_return_pred'] = (tmp['is_canceled_pred'] > threshold) * tmp['invoice_price_total']        
#         mse_score = mse(tmp['total_return'], tmp['total_return_pred'])
#         print('train set ratio: {}'.format(res_mse/res_mse_base))
    
        tmp = df_test.copy()[['invoice', 'invoice_price_total', 'is_canceled']].drop_duplicates()
        mse_score_max = sum(tmp.invoice_price_total * tmp.invoice_price_total) / (len(tmp)-1)
        ratio_lower += mse_score_max
    
        orders_test = df_test.copy()    
        orders_test['is_canceled_pred_1'] = model.predict(X_test.values) # 0,1
        orders_test['is_canceled_pred_2'] = model.predict_proba(X_test.values)[:, 1] # probrabilities
        orders_test['is_canceled_pred_3'] = orders_test['is_canceled_pred_2']*orders_test['price_total']/orders_test['invoice_price_total'] # probabilities weightedby price
        
        p_ratio_pred = sum(orders_test['is_canceled_pred_' + str(scenario['opt'])] > threshold)/len(orders_test)

        tmp = pd.merge(tmp, orders_test.groupby('invoice').agg(is_canceled_pred = ('is_canceled_pred_' + str(scenario['opt']), scenario['fun'])).reset_index(), how='left', on=['invoice'])            
        tmp['total_return'] = tmp['is_canceled'] * tmp['invoice_price_total']
        tmp['total_return_pred'] = (tmp['is_canceled_pred'] > threshold) * tmp['invoice_price_total']        
        mse_score = mse(tmp['total_return'], tmp['total_return_pred'])
        ratio_upper += mse_score

        fp_ratio_pred = sum(tmp['is_canceled_pred'] > threshold)/len(tmp)
        
        print('mse {} / mse base {} : ratio {} mean ratio {} : p_ratio_pred {} fp_ratio_pred {}'.format(mse_score, mse_score_max, mse_score/mse_score_max, ratio_upper/ratio_lower, p_ratio_pred, fp_ratio_pred))
        
        scores += [mse_score/mse_score_max]

        if data_valid is not None:
            data_valid['is_canceled_pred_1'] = model.predict(data_valid[feats].values) # 0,1
            data_valid['is_canceled_pred_2'] = model.predict_proba(data_valid[feats].values)[:, 1] # probrabilities
            data_valid['is_canceled_pred_3'] = data_valid['is_canceled_pred_2']*data_valid['price_total']/data_valid['invoice_price_total'] # probabilities weightedby price
            valid_preditions += [data_valid.copy()[['invoice', 'is_canceled_pred_' + str(scenario['opt'])]].rename(columns={'is_canceled_pred_' + str(scenario['opt']): 'is_canceled_pred'})]

        if data_to_predict is not None:
            data_to_predict['is_canceled_pred_1'] = model.predict(data_to_predict[feats].values) # 0,1
            data_to_predict['is_canceled_pred_2'] = model.predict_proba(data_to_predict[feats].values)[:, 1] # probrabilities
            data_to_predict['is_canceled_pred_3'] = data_to_predict['is_canceled_pred_2']*data_to_predict['price_total']/data_to_predict['invoice_price_total'] # probabilities weightedby price
            preditions += [data_to_predict.copy()[['invoice', 'is_canceled_pred_' + str(scenario['opt'])]].rename(columns={'is_canceled_pred_' + str(scenario['opt']): 'is_canceled_pred'})]

    scores_validation = None
    if data_valid is not None:
      w1 = 1/scores[0]
      w2 = 1/scores[1]
      w3 = 1/scores[2]
      w4 = 1/scores[3]
      w5 = 1/scores[4]
      w = w1+w2+w3+w4+w5
      w1 = w1/w
      w2 = w2/w
      w3 = w3/w
      w4 = w4/w
      w5 = w5/w

      tmp = data_valid.copy()[['invoice', 'invoice_price_total']].drop_duplicates()
      mse_score_max = sum(tmp.invoice_price_total * tmp.invoice_price_total) / (len(tmp)-1)

      tmp = data_valid.copy()[['invoice', 'description', 'invoice_price_total', 'is_canceled']]
      is_false = list(tmp[(tmp.description == 'adjust bad debt')].invoice.unique()) + [30477]
      is_true  = list(tmp[(tmp.description == 'amazon fee')].invoice.unique())
   
      tmp['is_canceled_pred'] = valid_preditions[0]['is_canceled_pred']*w1 + valid_preditions[1]['is_canceled_pred']*w2 + valid_preditions[2]['is_canceled_pred']*w3 + valid_preditions[3]['is_canceled_pred']*w4 + valid_preditions[4]['is_canceled_pred']*w5

      p_ratio = sum(tmp['is_canceled'] > threshold)/len(tmp)
      p_ratio_pred = sum(tmp['is_canceled_pred'] > threshold)/len(tmp)

      tmp = tmp.groupby('invoice').agg(is_canceled_pred = ('is_canceled_pred', scenario['fun']),
                                       is_canceled = ('is_canceled', np.mean),
                                       invoice_price_total = ('invoice_price_total', np.mean)).reset_index()
        
      tmp['total_return'] = tmp['is_canceled'] * tmp['invoice_price_total']

      fp_ratio = sum(tmp['is_canceled'] > threshold)/len(tmp)
      fp_ratio_pred = sum(tmp['is_canceled_pred'] > threshold)/len(tmp)

      tmp['is_canceled_pred'] = tmp.apply(lambda x: 0 if x['invoice'] in is_false else x['is_canceled_pred'], axis=1)
      tmp['is_canceled_pred'] = tmp.apply(lambda x: 1 if x['invoice'] in is_true else x['is_canceled_pred'], axis=1)

      tmp['total_return_pred'] = (tmp['is_canceled_pred'] > threshold) * tmp['invoice_price_total']        
      mse_score = mse(tmp['total_return'], tmp['total_return_pred'])
  
      scores_validation = mse_score/mse_score_max

      print('VALIDATION: mse {} / mse base {} : ratio {}'.format(mse_score, mse_score_max, scores_validation))
      print('VALIDATION: p {} p_pred {} : fp {} fp_pred {}'.format(p_ratio, p_ratio_pred, fp_ratio, fp_ratio_pred))

    submission = None
    if data_to_predict is not None:
      w1 = 1/scores[0]
      w2 = 1/scores[1]
      w3 = 1/scores[2]
      w4 = 1/scores[3]
      w5 = 1/scores[4]
      w = w1+w2+w3+w4+w5
      w1 = w1/w
      w2 = w2/w
      w3 = w3/w
      w4 = w4/w
      w5 = w5/w

      tmp = data_to_predict.copy()[['invoice', 'invoice_price_total']].drop_duplicates()
      mse_score_max = sum(tmp.invoice_price_total * tmp.invoice_price_total) / (len(tmp)-1)
  
      tmp = data_to_predict.copy()[['invoice', 'description', 'invoice_price_total']].drop_duplicates()
      is_false = list(tmp[(tmp.description == 'adjust bad debt')].invoice.unique()) + [30477]
      is_true  = list(tmp[(tmp.description == 'amazon fee')].invoice.unique())
   
      tmp['is_canceled_pred'] = preditions[0]['is_canceled_pred']*w1 + preditions[1]['is_canceled_pred']*w2 + preditions[2]['is_canceled_pred']*w3 + preditions[3]['is_canceled_pred']*w4 + preditions[4]['is_canceled_pred']*w5
      p_ratio_pred = sum(tmp['is_canceled_pred'] > threshold)/len(tmp)
      
      tmp = tmp.groupby('invoice').agg(is_canceled_pred = ('is_canceled_pred', scenario['fun']),
                                       invoice_price_total = ('invoice_price_total', np.mean)).reset_index()

      fp_ratio_pred = sum(tmp['is_canceled_pred'] > threshold)/len(tmp)

      tmp['is_canceled_pred'] = tmp.apply(lambda x: 0 if x['invoice'] in is_false else x['is_canceled_pred'], axis=1)
      tmp['is_canceled_pred'] = tmp.apply(lambda x: 1 if x['invoice'] in is_true else x['is_canceled_pred'], axis=1)

      tmp['total_return_pred'] = (tmp['is_canceled_pred'] > threshold) * tmp['invoice_price_total']        

      submission = tmp.copy()
      submission = submission.rename(columns={'total_return_pred': 'total_return'})

      print('SUBMISSION: ratio * mse base: {} * {} = {} : p_pred {} fp_pred {}'.format(ratio_upper/ratio_lower, mse_score_max, ratio_upper/ratio_lower*mse_score_max, p_ratio_pred, fp_ratio_pred))

    return scores, ratio_upper/ratio_lower, scores_validation, submission

In [0]:
best = {'model': {'colsample_bytree': 0.15000000000000002, 'eta': 0.025, 'eval_metric': ('auc', 'error', 'logloss'), 'gamma': 0.9, 'learning_rate': 0.28, 'max_depth': 10, 'min_child_weight': 10.0, 'n_estimators': 120, 'objective': 'binary:logistic', 'seed': 0, 'subsample': 0.79, 'tree_method': 'gpu_hist'}, 'threshold': 0.015}

model = xgb.XGBClassifier(**best['model'])

threshold = best['threshold']

feats = get_feats(train)

scenario={
    'opt': '3', 
    'fun': np.mean
}

### run

scores, ratio, scores_validation, submission = run_model(model, feats, train, data_to_predict=test, data_valid=valid, threshold=threshold, scenario=scenario)

mse 36004.667162455706 / mse base 1413979.3221247324 : ratio 0.025463361874594373 mean ratio 0.025463361874594373 : p_ratio_pred 0.016177882839624732 fp_ratio_pred 0.15557749259624876
mse 45406.13812185953 / mse base 1833752.1432910159 : ratio 0.024761327907908866 mean ratio 0.02506697556470965 : p_ratio_pred 0.016381200163812 fp_ratio_pred 0.15932872655478775
mse 49650.052176874764 / mse base 1317959.2996501562 : ratio 0.037671916113080316 mean ratio 0.028705592254296742 : p_ratio_pred 0.01536034942975006 fp_ratio_pred 0.15205371248025276
mse 39685.31819380141 / mse base 825381.321367404 : ratio 0.048081192494221936 mean ratio 0.03167202606781609 : p_ratio_pred 0.01557938037092637 fp_ratio_pred 0.1542259083728278
mse 25867.05001852385 / mse base 939887.3627723363 : ratio 0.0275214361242449 mean ratio 0.03105583399340595 : p_ratio_pred 0.01587919083321545 fp_ratio_pred 0.15165876777251186
VALIDATION: mse 6654.778736752044 / mse base 1585230.1557363835 : ratio 0.004197988987700473
VALID

In [0]:
submission[['invoice', 'total_return']].to_csv('submission_20200408_d_2.csv', index=False)

In [0]:
mse 40737.27563160426 / mse base 1657410.707915358 : ratio 0.02457886596065401 mean ratio 0.02457886596065401 : p_ratio_pred 0.01587264039362256 fp_ratio_pred 0.15596691610870422
mse 25525.10806073305 / mse base 1105614.3590499812 : ratio 0.02308680947547204 mean ratio 0.02398182502380046 : p_ratio_pred 0.016861771313704622 fp_ratio_pred 0.15714848365498227
mse 26226.63689472856 / mse base 1365189.0073213824 : ratio 0.01921099331600059 mean ratio 0.02240412413763844 : p_ratio_pred 0.01577784398430905 fp_ratio_pred 0.1553761323355652
mse 89764.2486250246 / mse base 915765.1153817634 : ratio 0.09802103958459234 mean ratio 0.03613283527921713 : p_ratio_pred 0.01548913043478261 fp_ratio_pred 0.15675462780622293
mse 25436.30419200403 / mse base 1293528.7919238745 : ratio 0.019664273691328074 mean ratio 0.03277148904701032 : p_ratio_pred 0.016006945718233734 fp_ratio_pred 0.1612839700669555
VALIDATION: mse 8535.446055506041 / mse base 1612787.2127774516 : ratio 0.005292357223496815
VALIDATION: p 0.019139997885083106 p_pred 0.018880439902713823 : fp 0.07529168029658707 fp_pred 0.08243375858684986
SUBMISSION: ratio * mse base: 0.03277148904701032 * 1894213.902757539 = 62076.21016691336 : p_pred 0.013055386955753437 fp_pred 0.14663981502200343

# ELI

In [0]:
feats = get_feats(train)

In [0]:
best = {'model': {'colsample_bytree': 0.15, 'eta': 0.025, 'eval_metric': ['auc', 'error', 'logloss'], 'gamma': 0.9, 'learning_rate': 0.28, 'max_depth': 10, 'min_child_weight': 10.0, 'n_estimators': 120, 'objective': 'binary:logistic', 'seed': 0, 'subsample': 0.79, 'tree_method': 'gpu_hist'}, 'threshold': 0.015}

model = xgb.XGBClassifier(**best['model'])

feats = get_feats(train, out)

X = train[feats].values
y = train['is_canceled'].values

# class weights
class_weights = list(class_weight.compute_class_weight('balanced',
                                                       np.unique(train['is_canceled']),
                                                       train['is_canceled']))
w_array = np.ones(y.shape[0], dtype = 'float')
for i, val in enumerate(y):
    w_array[i] = class_weights[val-1]

# model fit
model.fit(X, y, eval_metric=["auc","error","logloss"], sample_weight=w_array)

imp = PermutationImportance(model, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=feats, top=50)

Weight,Feature
0.0091 ± 0.0000,price_unit_sum_log
0.0083 ± 0.0001,invoice_price_total_log
0.0077 ± 0.0002,cnt_customer_cancel_log
0.0034 ± 0.0001,quantity_sum_log
0.0021 ± 0.0001,cnt_product_stock_cancel_2_log
0.0020 ± 0.0001,price_unit_min
0.0020 ± 0.0000,cnt_customer_orders
0.0019 ± 0.0001,price_unit_max
0.0017 ± 0.0001,quantity_mean
0.0017 ± 0.0000,quantity_max


In [0]:
feats_eli = ['cnt_customer_cancel','quantity_sum','price_unit_sum_log','quantity_sum_log','invoice_price_total_log','price_unit_sum','cnt_customer_not_cancel','quantity_max','customer_id','price_unit_max','cnt_product_cancel_2','quantity_mean','quantity_median','price_unit_mean','invoice_date_d','cnt_customer_orders','cnt_product_stock_cancel_2','price_unit_min','invoice_date_min','cnt_product_stock_cancel_2_log','invoice','price_unit_median','cnt_product_cancel','cnt_product_stock_cancel','cnt_product_stock_cancel_log','invoice_date_h','invoice_date_dow','country_cat','invoice_date_m','price_total_log','quantity_min','stock_code','cnt_product_stock_not_cancel_2_log','cnt_product_orders','cnt_product_not_cancel_2','description_len','description_cnt_space','cnt_product_orders_2','cnt_product_stock_not_cancel_log','quantity_stock','cnt_product_stock_not_cancel',]

In [0]:
0.0095 ± 0.0001 	cnt_customer_cancel
0.0065 ± 0.0001 	quantity_sum
0.0043 ± 0.0000 	quantity_sum_log
0.0040 ± 0.0001 	price_unit_sum_log
0.0040 ± 0.0000 	price_unit_sum
0.0033 ± 0.0001 	invoice_price_total_log
0.0023 ± 0.0001 	cnt_customer_not_cancel
0.0016 ± 0.0001 	customer_id
0.0016 ± 0.0001 	cnt_product_cancel_2
0.0016 ± 0.0001 	quantity_max
0.0013 ± 0.0001 	price_unit_max
0.0013 ± 0.0000 	quantity_median
0.0011 ± 0.0001 	price_unit_min
0.0011 ± 0.0000 	invoice_date_d
0.0011 ± 0.0001 	price_unit_mean
0.0010 ± 0.0001 	quantity_mean
0.0009 ± 0.0001 	cnt_product_cancel
0.0008 ± 0.0001 	invoice
0.0008 ± 0.0000 	cnt_customer_orders
0.0008 ± 0.0001 	invoice_date_min
0.0007 ± 0.0000 	cnt_product_stock_cancel_2
0.0007 ± 0.0001 	cnt_product_stock_cancel
0.0006 ± 0.0000 	price_unit_median
0.0006 ± 0.0001 	cnt_product_stock_cancel_2_log
0.0006 ± 0.0001 	cnt_product_stock_cancel_log
0.0006 ± 0.0001 	invoice_date_m
0.0005 ± 0.0001 	invoice_date_h
0.0004 ± 0.0000 	invoice_date_dow
0.0003 ± 0.0000 	country_cat
0.0002 ± 0.0000 	quantity_min
0.0002 ± 0.0000 	cnt_product_orders
0.0002 ± 0.0001 	cnt_product_stock_not_cancel_2_log
0.0001 ± 0.0000 	stock_code
0.0001 ± 0.0000 	cnt_product_stock_not_cancel_log
0.0001 ± 0.0000 	cnt_product_stock_not_cancel
0.0001 ± 0.0000 	cnt_product_stock_not_cancel_2
0.0001 ± 0.0000 	quantity
0.0001 ± 0.0000 	cnt_product_orders_2
0.0001 ± 0.0000 	invoice_date_y
0.0001 ± 0.0000 	stock_code_2
0.0001 ± 0.0000 	cnt_product_not_cancel
0.0001 ± 0.0000 	quantity_log
0.0001 ± 0.0000 	price_cheap_exp_mean
0.0001 ± 0.0000 	description_len
0.0001 ± 0.0000 	description_cnt_space
0.0001 ± 0.0000 	price_total
0.0001 ± 0.0000 	quantity_stock
0.0000 ± 0.0000 	quantity_stock_log
0.0000 ± 0.0000 	cnt_product_not_cancel_2
0.0000 ± 0.0000 	price_unit 

# HYPER

In [0]:
# hyperopt
# feats = get_feats(train)
feats = feats_eli

def obj_func(params):
  print("Training with params: ")
  print(params)

  scores, ratio, scores_validation, submission = run_model(xgb.XGBClassifier(**params['model']), feats, train, data_valid=valid, threshold=params['threshold'], scenario={'opt': '3', 'fun': np.mean})
  print('MSE: {} ({}) - {} + {}'.format(np.abs(np.mean(scores)), np.std(scores), ratio, scores_validation))
  print('---------------- \n')

  return {'loss': scores_validation, 'status': STATUS_OK}

# space
f_params = {'model': {
    'max_depth' : hp.choice('max_depth', range(1, 16, 1)),
    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.3, 0.02),
    'n_estimators' : hp.choice('n_estimators', range(50, 200, 10)),
    'gamma' : hp.quniform('gamma', 0, 1, 0.1),
    'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
    'min_child_weight' : hp.quniform('min_child_weight', 1, 80, 5),
    'subsample' : hp.quniform('subsample', 0.1, 1, 0.01),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1, 0.05),
    'seed': 0,
    'tree_method': 'gpu_hist',
    'objective': 'binary:logistic', 
    'eval_metric': ["auc","error","logloss"],
},
    'threshold': hp.quniform('threshold', 0.001, 0.030, 0.001)
}

# run
best = fmin(obj_func, f_params, algo=tpe.suggest, max_evals=30, return_argmin=False)

print('---------------- \n')

best

Training with params: 
{'model': {'colsample_bytree': 0.75, 'eta': 0.1, 'eval_metric': ('auc', 'error', 'logloss'), 'gamma': 0.4, 'learning_rate': 0.1, 'max_depth': 15, 'min_child_weight': 15.0, 'n_estimators': 140, 'objective': 'binary:logistic', 'seed': 0, 'subsample': 0.64, 'tree_method': 'gpu_hist'}, 'threshold': 0.016}
mse 24101.0416742197 / mse base 1657410.707915358 : ratio 0.014541381661841244 mean ratio 0.014541381661841244 : p_ratio_pred 0.0152221223447036 fp_ratio_pred 0.15222528554549036
mse 36040.9476778284 / mse base 1105614.3590499812 : ratio 0.032598118306637426 mean ratio 0.021766718684931335 : p_ratio_pred 0.016554333688069054 fp_ratio_pred 0.15714848365498227
mse 20640.380649689065 / mse base 1365189.0073213824 : ratio 0.015119064495096732 mean ratio 0.019568357780887332 : p_ratio_pred 0.01596404985351805 fp_ratio_pred 0.15242221346987003
mse 90149.01504585512 / mse base 915765.1153817634 : ratio 0.09844119800116416 mean ratio 0.03388820187793573 : p_ratio_pred 0.015

{'model': {'colsample_bytree': 0.2,
  'eta': 0.225,
  'eval_metric': ('auc', 'error', 'logloss'),
  'gamma': 0.30000000000000004,
  'learning_rate': 0.28,
  'max_depth': 8,
  'min_child_weight': 5.0,
  'n_estimators': 120,
  'objective': 'binary:logistic',
  'seed': 0,
  'subsample': 0.54,
  'tree_method': 'gpu_hist'},
 'threshold': 0.012}

In [0]:
# import json
# with open('best.json', 'w') as f:
#   f.write(json.dumps(best))

In [0]:
best = {'model': {'colsample_bytree': 0.2,
  'eta': 0.225,
  'eval_metric': ['auc', 'error', 'logloss'],
  'gamma': 0.30000000000000004,
  'learning_rate': 0.28,
  'max_depth': 8,
  'min_child_weight': 5.0,
  'n_estimators': 120,
  'objective': 'binary:logistic',
  'seed': 0,
  'subsample': 0.54,
  'tree_method': 'gpu_hist'},
 'threshold': 0.012}

In [0]:
# model eli + best

model = xgb.XGBClassifier(**best['model'])

feats = feats_eli

threshold = best['threshold']

scenario={
    'opt': '3', 
    'fun': np.mean
}

### run

scores, ratio, scores_validation, submission = run_model(model, feats, train, data_to_predict=test, data_valid=valid, threshold=threshold, scenario=scenario)

mse 25883.152202387362 / mse base 1657410.707915358 : ratio 0.015616619392390932 mean ratio 0.015616619392390932 : p_ratio_pred 0.016369399630978852 fp_ratio_pred 0.16029933044505712
mse 36165.90686542749 / mse base 1105614.3590499812 : ratio 0.03271114070597246 mean ratio 0.02245692947547668 : p_ratio_pred 0.01744117299278704 fp_ratio_pred 0.16266246553761324
mse 145347.45452372852 / mse base 1365189.0073213824 : ratio 0.10646690952259619 mean ratio 0.05023879814841666 : p_ratio_pred 0.016522667461145044 fp_ratio_pred 0.15813312327688067
mse 95298.75082835174 / mse base 915765.1153817634 : ratio 0.10406462227885113 mean ratio 0.06001120405887118 : p_ratio_pred 0.015926275992438562 fp_ratio_pred 0.1577392674281213
mse 18663.64061478237 / mse base 1293528.7919238745 : ratio 0.014428469417386377 mean ratio 0.05070745566997029 : p_ratio_pred 0.016924074006138646 fp_ratio_pred 0.1636471051595116
VALIDATION: mse 7320.939172447348 / mse base 1612787.2127774516 : ratio 0.004539308790674027
VA

In [0]:
submission[['invoice', 'total_return']].to_csv('submission_20200408_xgboost_hyperopt_eli.csv', index=False)

In [0]:
# model #1

def run_model_v2(model, feats, data, data_to_predict=None, data_valid=None, threshold=0.02, scenario={'opt': '3', 'fun': np.mean}):
#     data = train.copy()
#     data_test = test.copy()

    X_invoice = data[['invoice', 'is_canceled']].drop_duplicates().reset_index(drop=True)
    
    scores = []
    ratio_upper = 0
    ratio_lower = 0
    preditions = []
    valid_preditions = []
    
    skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
    for train_invoice_index, test_invoice_index in skf.split(X_invoice[['invoice']], X_invoice['is_canceled'] == True):
        train_invoice_list = list(X_invoice.iloc[train_invoice_index].invoice)
        test_invoice_list = list(X_invoice.iloc[test_invoice_index].invoice)

        df_train, df_test = data[data.invoice.isin(train_invoice_list)], data[data.invoice.isin(test_invoice_list)]

        X_train, X_test = df_train[feats], df_test[feats]
        y_train, y_test = df_train['is_canceled'], df_test['is_canceled']

        # class weights
        class_weights = list(class_weight.compute_class_weight('balanced',
                                                               np.unique(df_train['is_canceled']),
                                                               df_train['is_canceled']))

        w_array = np.ones(y_train.shape[0], dtype = 'float')
        for i, val in enumerate(y_train):
            w_array[i] = class_weights[val-1]

        # model fit
        
        model.fit(X_train.values, y_train, eval_metric=["auc","error","logloss"], sample_weight=w_array)

#         tmp = df_train.copy()[['invoice', 'invoice_price_total', 'is_canceled']].drop_duplicates()
#         mse_score_max = sum(tmp.invoice_price_total * tmp.invoice_price_total) / (len(tmp)-1)
#         orders_train = df_train.copy()    
#         orders_train['is_canceled_pred_1'] = model.predict(X_train.values) # 0,1
#         orders_train['is_canceled_pred_2'] = model.predict_proba(X_train.values)[:, 1] # probrabilities
#         orders_train['is_canceled_pred_3'] = orders_train['is_canceled_pred_2']*orders_train['price_total']/orders_train['invoice_price_total'] # probabilities weightedby price
#         tmp = pd.merge(tmp, orders_train.groupby('invoice').agg(is_canceled_pred = ('is_canceled_pred_' + str(scenario['opt']), scenario['fun'])).reset_index(), how='left', on=['invoice'])            
#         tmp['total_return'] = tmp['is_canceled'] * tmp['invoice_price_total']
#         tmp['total_return_pred'] = (tmp['is_canceled_pred'] > threshold) * tmp['invoice_price_total']        
#         mse_score = mse(tmp['total_return'], tmp['total_return_pred'])
#         print('train set ratio: {}'.format(res_mse/res_mse_base))
    
        tmp = df_test.copy()[['invoice', 'invoice_price_total', 'is_canceled']].drop_duplicates()
        mse_score_max = sum(tmp.invoice_price_total * tmp.invoice_price_total) / (len(tmp)-1)
        ratio_lower += mse_score_max
    
        orders_test = df_test.copy()    
        orders_test['is_canceled_pred_1'] = model.predict(X_test.values) # 0,1
        orders_test['is_canceled_pred_2'] = model.predict_proba(X_test.values)[:, 1] # probrabilities
        orders_test['is_canceled_pred_3'] = orders_test['is_canceled_pred_2']*orders_test['price_total']/orders_test['invoice_price_total'] # probabilities weightedby price
        
        p_ratio_pred = sum(orders_test['is_canceled_pred_' + str(scenario['opt'])] > threshold)/len(orders_test)

        tmp = pd.merge(tmp, orders_test.groupby('invoice').agg(is_canceled_pred = ('is_canceled_pred_' + str(scenario['opt']), scenario['fun'])).reset_index(), how='left', on=['invoice'])            
        tmp['total_return'] = tmp['is_canceled'] * tmp['invoice_price_total']
        tmp['total_return_pred'] = (tmp['is_canceled_pred'] > threshold) * tmp['invoice_price_total']        
        mse_score = mse(tmp['total_return'], tmp['total_return_pred'])
        ratio_upper += mse_score

        fp_ratio_pred = sum(tmp['is_canceled_pred'] > threshold)/len(tmp)
        
        print('mse {} / mse base {} : ratio {} mean ratio {} : p_ratio_pred {} fp_ratio_pred {}'.format(mse_score, mse_score_max, mse_score/mse_score_max, ratio_upper/ratio_lower, p_ratio_pred, fp_ratio_pred))
        
        scores += [mse_score/mse_score_max]

        if data_valid is not None:
            data_valid['is_canceled_pred_1'] = model.predict(data_valid[feats].values) # 0,1
            data_valid['is_canceled_pred_2'] = model.predict_proba(data_valid[feats].values)[:, 1] # probrabilities
            data_valid['is_canceled_pred_3'] = data_valid['is_canceled_pred_2']*data_valid['price_total']/data_valid['invoice_price_total'] # probabilities weightedby price
            valid_preditions += [data_valid.copy()[['invoice', 'is_canceled_pred_' + str(scenario['opt'])]].rename(columns={'is_canceled_pred_' + str(scenario['opt']): 'is_canceled_pred'})]

        if data_to_predict is not None:
            data_to_predict['is_canceled_pred_1'] = model.predict(data_to_predict[feats].values) # 0,1
            data_to_predict['is_canceled_pred_2'] = model.predict_proba(data_to_predict[feats].values)[:, 1] # probrabilities
            data_to_predict['is_canceled_pred_3'] = data_to_predict['is_canceled_pred_2']*data_to_predict['price_total']/data_to_predict['invoice_price_total'] # probabilities weightedby price
            preditions += [data_to_predict.copy()[['invoice', 'is_canceled_pred_' + str(scenario['opt'])]].rename(columns={'is_canceled_pred_' + str(scenario['opt']): 'is_canceled_pred'})]

    scores_validation = None
    if data_valid is not None:
      w1 = 1/scores[0]
      w2 = 1/scores[1]
      w3 = 1/scores[2]
      w4 = 1/scores[3]
      w5 = 1/scores[4]
      w = w1+w2+w3+w4+w5
      w1 = w1/w
      w2 = w2/w
      w3 = w3/w
      w4 = w4/w
      w5 = w5/w

      tmp = data_valid.copy()[['invoice', 'invoice_price_total']].drop_duplicates()
      mse_score_max = sum(tmp.invoice_price_total * tmp.invoice_price_total) / (len(tmp)-1)

      tmp = data_valid.copy()[['invoice', 'description', 'invoice_price_total', 'is_canceled']]
      is_false = list(tmp[(tmp.description == 'adjust bad debt')].invoice.unique()) + [30477]
      is_true  = list(tmp[(tmp.description == 'amazon fee')].invoice.unique())
   
      tmp['is_canceled_pred'] = valid_preditions[0]['is_canceled_pred']*w1 + valid_preditions[1]['is_canceled_pred']*w2 + valid_preditions[2]['is_canceled_pred']*w3 + valid_preditions[3]['is_canceled_pred']*w4 + valid_preditions[4]['is_canceled_pred']*w5

      p_ratio = sum(tmp['is_canceled'] > threshold)/len(tmp)
      p_ratio_pred = sum(tmp['is_canceled_pred'] > threshold)/len(tmp)

      tmp = tmp.groupby('invoice').agg(is_canceled_pred = ('is_canceled_pred', scenario['fun']),
                                       is_canceled = ('is_canceled', np.mean),
                                       invoice_price_total = ('invoice_price_total', np.mean)).reset_index()
        
      tmp['total_return'] = tmp['is_canceled'] * tmp['invoice_price_total']

      fp_ratio = sum(tmp['is_canceled'] > threshold)/len(tmp)
      fp_ratio_pred = sum(tmp['is_canceled_pred'] > threshold)/len(tmp)

      tmp['is_canceled_pred'] = tmp.apply(lambda x: 0 if x['invoice'] in is_false else x['is_canceled_pred'], axis=1)
      tmp['is_canceled_pred'] = tmp.apply(lambda x: 1 if x['invoice'] in is_true else x['is_canceled_pred'], axis=1)

      tmp['total_return_pred'] = (tmp['is_canceled_pred'] > threshold) * tmp['invoice_price_total']        
      mse_score = mse(tmp['total_return'], tmp['total_return_pred'])
  
      scores_validation = mse_score/mse_score_max

      print('VALIDATION: mse {} / mse base {} : ratio {}'.format(mse_score, mse_score_max, scores_validation))
      print('VALIDATION: p {} p_pred {} : fp {} fp_pred {}'.format(p_ratio, p_ratio_pred, fp_ratio, fp_ratio_pred))

    submission = None
    if data_to_predict is not None:
      w1 = 1/scores[0]
      w2 = 1/scores[1]
      w3 = 1/scores[2]
      w4 = 1/scores[3]
      w5 = 1/scores[4]
      w = w1+w2+w3+w4+w5
      w1 = w1/w
      w2 = w2/w
      w3 = w3/w
      w4 = w4/w
      w5 = w5/w

      tmp = data_to_predict.copy()[['invoice', 'invoice_price_total']].drop_duplicates()
      mse_score_max = sum(tmp.invoice_price_total * tmp.invoice_price_total) / (len(tmp)-1)
  
      tmp = data_to_predict.copy()[['invoice', 'description', 'invoice_price_total']].drop_duplicates()
      is_false = list(tmp[(tmp.description == 'adjust bad debt')].invoice.unique()) + [30477]
      is_true  = list(tmp[(tmp.description == 'amazon fee')].invoice.unique())
   
      tmp['is_canceled_pred'] = preditions[0]['is_canceled_pred']*w1 + preditions[1]['is_canceled_pred']*w2 + preditions[2]['is_canceled_pred']*w3 + preditions[3]['is_canceled_pred']*w4 + preditions[4]['is_canceled_pred']*w5
      p_ratio_pred = sum(tmp['is_canceled_pred'] > threshold)/len(tmp)
      
      tmp = tmp.groupby('invoice').agg(is_canceled_pred = ('is_canceled_pred', scenario['fun']),
                                       invoice_price_total = ('invoice_price_total', np.mean)).reset_index()

      fp_ratio_pred = sum(tmp['is_canceled_pred'] > threshold)/len(tmp)

      tmp['is_canceled_pred'] = tmp.apply(lambda x: 0 if x['invoice'] in is_false else x['is_canceled_pred'], axis=1)
      tmp['is_canceled_pred'] = tmp.apply(lambda x: 1 if x['invoice'] in is_true else x['is_canceled_pred'], axis=1)

      tmp['total_return_pred'] = (tmp['is_canceled_pred'] > threshold) * tmp['invoice_price_total']        

      submission = tmp.copy()
      submission = submission.rename(columns={'total_return_pred': 'total_return'})

      print('SUBMISSION: ratio * mse base: {} * {} = {} : p_pred {} fp_pred {}'.format(ratio_upper/ratio_lower, mse_score_max, ratio_upper/ratio_lower*mse_score_max, p_ratio_pred, fp_ratio_pred))

    return scores, ratio_upper/ratio_lower, scores_validation, submission

In [0]:
best = {'model': {'bagging_freq': 10, 'bagging_seed': 2020, 'boost_from_average': False, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.7207837526465946, 'data_random_seed': 2020, 'drop_seed': 2020, 'feature_fraction_seed': 2020, 'is_unbalance': True, 'learning_rate': 0.19565655054976303, 'max_bin': 150, 'max_depth': 12, 'metric': 'auc', 'min_child_samples': 260.0, 'min_data_in_leaf': 8, 'num_leaves': 180, 'objective': 'binary', 'reg_alpha': 0.9141189048912121, 'reg_lambda': 0.28623237458465467, 'save_binary': True, 'seed': 2020, 'verbose': 1}, 'threshold': 0.034}

model = lgb.LGBMClassifier(**best['model'])

feats = get_feats(train, out)

X = train[feats].values
y = train['is_canceled'].values

# class weights
class_weights = list(class_weight.compute_class_weight('balanced',
                                                       np.unique(train['is_canceled']),
                                                       train['is_canceled']))
w_array = np.ones(y.shape[0], dtype = 'float')
for i, val in enumerate(y):
    w_array[i] = class_weights[val-1]

# model fit
model.fit(X, y, eval_metric=["auc","error","logloss"], sample_weight=w_array)

imp = PermutationImportance(model, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=feats, top=50)

Weight,Feature
0.0129 ± 0.0002,cnt_customer_cancel_log
0.0095 ± 0.0001,quantity_sum_log
0.0038 ± 0.0001,price_unit_sum_log
0.0029 ± 0.0002,cnt_product_cancel_2
0.0016 ± 0.0001,invoice_price_total_log
0.0015 ± 0.0000,cnt_customer_not_cancel
0.0008 ± 0.0000,invoice_date_min
0.0008 ± 0.0001,cnt_product_stock_cancel_2_log
0.0006 ± 0.0000,customer_id
0.0003 ± 0.0000,quantity_min


In [0]:
feats_eli = ['cnt_customer_cancel_log','quantity_sum_log','price_unit_sum_log','cnt_product_cancel_2','invoice_price_total_log','cnt_customer_not_cancel','invoice_date_min','cnt_product_stock_cancel_2_log','customer_id','quantity_min','price_unit_min','invoice_date_h','cnt_product_cancel','price_unit_max','invoice','price_total_log','invoice_date_d','invoice_date_m','quantity_mean','cnt_product_stock_cancel_log','invoice_date_dow','quantity_median','price_unit_median','price_unit_mean','country_cat','stock_code','price_cheap_exp_median','cnt_customer_orders','price_cheap_exp_mean','stock_code_2','cnt_product_orders','cat_pack','cnt_product_orders_2',]

In [0]:
# hyperopt
# feats = get_feats(train, out)
feats = feats_eli

def obj_func(params):
  print("Training with params: ")
  print(params)

  scores, ratio, scores_validation, submission = run_model_v2(lgb.LGBMClassifier(**params['model']), feats, train, data_valid=valid, threshold=params['threshold'], scenario={'opt': '3', 'fun': np.mean})
  print('MSE: {} ({}) - {} + {}'.format(np.abs(np.mean(scores)), np.std(scores), ratio, scores_validation))
  print('---------------- \n')

  return {'loss': scores_validation, 'status': STATUS_OK}

# space
f_params = {'model': {
    'class_weight': hp.choice('class_weight', [None, 'balanced']),
    # 'boosting_type': hp.choice('boosting_type', 
    #                            [{'boosting_type': 'gbdt', 
    #                                 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}, 
    #                              {'boosting_type': 'dart', 
    #                                  'subsample': hp.uniform('dart_subsample', 0.5, 1)},
    #                              {'boosting_type': 'goss'}]),
    'num_leaves': hp.choice('num_leaves', range(30, 200, 10)),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    # 'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000, dtype=int),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),

    'max_bin': hp.choice('max_bin', range(50, 200, 10)),
    'min_data_in_leaf': hp.choice('min_data_in_leaf', range(5, 30, 1)),
    # 'min_sum_hessian_in_leaf': 0.00245,
    # 'bagging_fraction': 1.0,
    'bagging_freq': hp.choice('bagging_freq', range(1, 15, 1)),
    # 'feature_fraction': 0.05,
    # 'lambda_l1': 4.972,
    # 'lambda_l2': 2.276,
    # 'min_gain_to_split': 0.65,
    'max_depth': hp.choice('max_depth', range(1, 16, 1)),

    'save_binary': True,
    'seed': 2020,
    'feature_fraction_seed': 2020,
    'bagging_seed': 2020,
    'drop_seed': 2020,
    'data_random_seed': 2020,
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'verbose': 1,
    'metric': "auc",
    'is_unbalance': True,
    'boost_from_average': False,
},
    'threshold': hp.quniform('threshold', 0.1, 0.25, 0.005)
}

# run
best = fmin(obj_func, f_params, algo=tpe.suggest, max_evals=50, return_argmin=False)

print('---------------- \n')

best

Training with params: 
{'model': {'bagging_freq': 10, 'bagging_seed': 2020, 'boost_from_average': False, 'boosting_type': 'gbdt', 'class_weight': 'balanced', 'colsample_bytree': 0.700873410133455, 'data_random_seed': 2020, 'drop_seed': 2020, 'feature_fraction_seed': 2020, 'is_unbalance': True, 'learning_rate': 0.09928038996568483, 'max_bin': 80, 'max_depth': 5, 'metric': 'auc', 'min_child_samples': 365.0, 'min_data_in_leaf': 12, 'num_leaves': 60, 'objective': 'binary', 'reg_alpha': 0.14054543315286472, 'reg_lambda': 0.171761841288958, 'save_binary': True, 'seed': 2020, 'verbose': 1}, 'threshold': 0.14}
  0%|          | 0/50 [00:00<?, ?it/s, best loss: ?]


NameError: ignored

In [0]:
import json
with open('best_2.json', 'w') as f:
  f.write(json.dumps(best))

In [0]:
best = \
{'model': {'bagging_freq': 5,
  'bagging_seed': 2020,
  'boost_from_average': False,
  'boosting_type': 'gbdt',
  'class_weight': None,
  'colsample_bytree': 0.701053176860787,
  'data_random_seed': 2020,
  'drop_seed': 2020,
  'feature_fraction_seed': 2020,
  'is_unbalance': True,
  'learning_rate': 0.12771712520652634,
  'max_bin': 170,
  'max_depth': 10,
  'metric': 'auc',
  'min_child_samples': 335.0,
  'min_data_in_leaf': 26,
  'num_leaves': 150,
  'objective': 'binary',
  'reg_alpha': 0.6456603541982964,
  'reg_lambda': 0.2628732832617849,
  'save_binary': True,
  'seed': 2020,
  'verbose': 1},
 'threshold': 0.12}

In [0]:
# model eli + best

model = lgb.LGBMClassifier(**best['model'])

feats = feats_eli

threshold = best['threshold']

scenario={
    'opt': '3', 
    'fun': np.mean
}

### run

scores, ratio, scores_validation, submission = run_model_v2(model, feats, train, data_to_predict=test, data_valid=valid, threshold=threshold, scenario=scenario)

mse 22107.687202022273 / mse base 1657410.707915358 : ratio 0.01333868973842257 mean ratio 0.01333868973842257 : p_ratio_pred 0.013093154184605195 fp_ratio_pred 0.15478534856242615
mse 35549.59729100755 / mse base 1105614.3590499812 : ratio 0.032153704408790576 mean ratio 0.02086744893572589 : p_ratio_pred 0.013680974340782784 fp_ratio_pred 0.15636077195746356
mse 146755.0525319985 / mse base 1365189.0073213824 : ratio 0.10749797408634608 mean ratio 0.049515924646021885 : p_ratio_pred 0.013816475495307613 fp_ratio_pred 0.15419456478928711
mse 92002.25449555025 / mse base 915765.1153817634 : ratio 0.10046490410065077 mean ratio 0.0587660218994798 : p_ratio_pred 0.013445179584120982 fp_ratio_pred 0.15833005120126034
mse 27355.132775675644 / mse base 1293528.7919238745 : ratio 0.02114767985565297 mean ratio 0.051087860597045585 : p_ratio_pred 0.014111547256563581 fp_ratio_pred 0.16010240252067742
VALIDATION: mse 33176.38174998268 / mse base 1612787.2127774516 : ratio 0.02057083630570717
V

In [0]:
submission[['invoice', 'total_return']].to_csv('submission_20200408_lgbm_hyperopt_eli_final.csv', index=False)

In [0]:
mse 35913.58464171226 / mse base 1413979.3221247324 : ratio 0.025398946137165777 mean ratio 0.025398946137165777 : p_ratio_pred 0.01960060082243727 fp_ratio_pred 0.18302073050345508
mse 37601.65312482367 / mse base 1833752.1432910159 : ratio 0.02050530834409299 mean ratio 0.022635873239330487 : p_ratio_pred 0.018585435185854354 fp_ratio_pred 0.1788746298124383
mse 57256.690000476374 / mse base 1317959.2996501562 : ratio 0.043443443219889105 mean ratio 0.028642309454596764 : p_ratio_pred 0.0184421256976462 fp_ratio_pred 0.1796998420221169
mse 46602.30779920009 / mse base 825381.321367404 : ratio 0.05646154885355819 mean ratio 0.03290147724282459 : p_ratio_pred 0.019125521578269248 fp_ratio_pred 0.18305687203791468
mse 169852.20341477386 / mse base 939887.3627723363 : ratio 0.18071548798546427 mean ratio 0.054845784713492884 : p_ratio_pred 0.01881454236808601 fp_ratio_pred 0.1779225908372828
VALIDATION: mse 22772.978063982544 / mse base 1585230.1557363835 : ratio 0.014365723476540769
VALIDATION: p 0.019322649798602234 p_pred 0.02030320217644175 : fp 0.07657706481682203 fp_pred 0.09299804899197919
SUBMISSION: ratio * mse base: 0.054845784713492884 * 1894213.902757539 = 103889.64791194513 : p_pred 0.015876979340335134 fp_pred 0.1712538226299694